# 1.6 Ajax
其实ajax是前端技术,用的也基本是javascript.本来不该在这边说,不过现在流行这个,那就一道说了.

所谓ajax是一种即时数据交换技术,通过js和xml让页面可以局部刷新.



## CSRF 保护

要对所有视图函数启用 CSRF 保护，你需要启用 CsrfProtect 模块:

    from flask_wtf.csrf import CsrfProtect

    CsrfProtect(app)


如果模板中有表单，你不需要做任何事。与之前一样:

    <form method="post" action="/">
        {{ form.csrf_token }}
    </form>

但如果模板中没有表单，你仍需要 CSRF 令牌:

    <form method="post" action="/">
        <input type="hidden" name="csrf_token" value="{{ csrf_token() }}" />
    </form>

无论何时未通过 CSRF 验证，都会返回 400 响应。你可以自定义这个错误响应:

    @csrf.error_handler
    def csrf_error(reason):
        return render_template('csrf_error.html', reason=reason), 400

将某些视图函数除外的途径:

    @csrf.exempt
    @app.route('/foo', methods=('GET', 'POST'))
    def my_handler():
        # ...
        return 'ok'




## AJAX

通过 AJAX 发送 POST 请求并不需要表单。

假设你已经使用了 CsrfProtect(app) ，你可以通过 {{ csrf_token() }} 获取 CSRF 令牌。这个方法在每个模板中都可以使用，你并不需要担心在没有表单时如何渲染 CSRF 令牌字段。

我们推荐的方式是在 <meta> 标签中渲染 CSRF 令牌:

    <meta name="csrf-token" content="{{ csrf_token() }}">


在定义AJAX时要写入:

    var csrftoken = $('meta[name=csrf-token]').attr('content')

    $.ajaxSetup({
        beforeSend: function(xhr, settings) {
            if (!/^(GET|HEAD|OPTIONS|TRACE)$/i.test(settings.type)) {
                xhr.setRequestHeader("X-CSRFToken", csrftoken)
            }
        }
    })



> 例利用ajax修改主页,点击切换图片:

In [7]:
%%writefile ../codes/ajax_simple/manager.py
#coding:UTF-8
import os
import sys

root = os.path.dirname(__file__)

sys.path.insert(0, os.path.join(root, 'site-packages'))

from app import manager

if __name__ == '__main__':
    manager.run()

Overwriting ../codes/ajax_simple/manager.py


In [12]:
%%writefile ../codes/ajax_simple/app.py
#coding:UTF-8
from flask import Flask,jsonify, render_template, request,url_for
from flask.ext.script import Manager

app = Flask(__name__)
app.config['SECRET_KEY'] = 'hard to guess string'
app.debug = True

manager = Manager(app)



@app.route('/',methods = ["GET","POST"])
def hello():
    return render_template('index.html')

@app.route('/ajax',methods = ["GET"])
def ajax():
    return jsonify({"hello":"hello world !"})

Overwriting ../codes/ajax_simple/app.py


In [8]:
%%writefile ../codes/ajax_simple/templates/layout.html
<!doctype html>
<title>jQuery Example</title>
<script type=text/javascript src="{{
  url_for('static', filename='js/jquery.min.js') }}"></script>
<script type="text/javascript">
  var $SCRIPT_ROOT = {{ request.script_root|tojson|safe }};
</script>
{% block body %}{% endblock %}

Overwriting ../codes/ajax_simple/templates/layout.html


In [14]:
%%writefile ../codes/ajax_simple/templates/index.html
{% extends "layout.html" %}
{% block body %}
<div id="result"><p>here is the result</p></div>
<button id="hello">
<h1>jQuery Example</h1>
</button>
<script type="text/javascript">
    $("#hello").click(function() {
        $.getJSON($SCRIPT_ROOT + '/ajax',
                function(data) {
                    $('#result').text(data.hello)
                })
        return false
    })
    
</script>

{% endblock %}

Overwriting ../codes/ajax_simple/templates/index.html


In [24]:
!python3 ../codes/helloworld/manager.py runserver

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger pin code: 258-121-533
127.0.0.1 - - [20/Jan/2016 00:17:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2016 00:17:39] "GET /changimg HTTP/1.1" 200 -
^C


## Get方式接收传递参数

如果使用get方式传递参数,后端获取参数要使用`request`变量,具体是:

`password = request.args.get('password', 0, type=str)`

其中get方法的第一位是传过来json的字段,第二位是默认值,第三位是给予的类型

## Post方式接收传递参数

如果使用post方式传递参数,后端获取参数要使用request变量,具体是:
code = request.values.get('code')
其中get方法的第一位是传过来json的字段,第二位是默认值,第三位是给予的类型和之前一样


## 总结

在flask中使用ajax最重要的是注意 CSRF 保护的问题,其他的和一般用ajax是一样的.

